In [4]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from datetime import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pandas.plotting import register_matplotlib_converters

import statsmodels.api as sm
from statsmodels.tsa.api import Holt

import acquire as a
import prepare as p

In [6]:
df = pd.read_csv('saas.csv')

In [7]:
df.head()

,Month_Invoiced,Customer_Id,Invoice_Id,Subscription_Type,Amount
0,2014-01-31,1000000,5000000,0.0,0.0
1,2014-01-31,1000001,5000001,2.0,10.0
2,2014-01-31,1000002,5000002,0.0,0.0
3,2014-01-31,1000003,5000003,0.0,0.0
4,2014-01-31,1000004,5000004,1.0,5.0


In [9]:
df.shape

(790458, 5)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790458 entries, 0 to 790457
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Month_Invoiced     790458 non-null  object 
 1   Customer_Id        790458 non-null  int64  
 2   Invoice_Id         790458 non-null  int64  
 3   Subscription_Type  790458 non-null  float64
 4   Amount             790458 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 30.2+ MB


In [8]:
df.isnull().sum().value_counts

<bound method IndexOpsMixin.value_counts of Month_Invoiced       0
Customer_Id          0
Invoice_Id           0
Subscription_Type    0
Amount               0
dtype: int64>

In [11]:
df['Month_Invoice'] = pd.to_datetime(df.Month_Invoiced)

In [12]:
df.set_index(df.Month_Invoiced)

,Month_Invoiced,Customer_Id,Invoice_Id,Subscription_Type,Amount,Month_Invoice
Month_Invoiced,,,,,,
2014-01-31,2014-01-31,1000000,5000000,0.0,0.0,2014-01-31
2014-01-31,2014-01-31,1000001,5000001,2.0,10.0,2014-01-31
2014-01-31,2014-01-31,1000002,5000002,0.0,0.0,2014-01-31
2014-01-31,2014-01-31,1000003,5000003,0.0,0.0,2014-01-31
2014-01-31,2014-01-31,1000004,5000004,1.0,5.0,2014-01-31
...,...,...,...,...,...,...
2017-12-31,2017-12-31,1029395,5790453,0.0,0.0,2017-12-31
2017-12-31,2017-12-31,1029396,5790454,0.0,0.0,2017-12-31
2017-12-31,2017-12-31,1029397,5790455,3.0,15.0,2017-12-31


In [14]:
train_size = int(len(df) * .5)
validate_size = int(len(df) * .3)
test_size = int(len(df) - train_size - validate_size)
validate_end_index = train_size + validate_size

# split into train, validation, test
train = df[: train_size]
validate = df[train_size : validate_end_index]
test = df[validate_end_index : ]


In [22]:
def time_plots(target):  
    periods = ['D', 'W', 'M','Q', 'Y']
    period_labels = ['Daily', 'Weekly', 
                     'Monthly', 'Quarterly', 'Yearly']
    alphas = [.2, .35, .5, .65, .8]
    for period, period_label, alpha in zip(periods, period_labels, alphas):
        train[target].resample(per).mean().plot(alpha=alpha, 
                                                figsize=(12,8), 
                                                label=period_label)
    plt.title('Resampled Averages for Various Periods')
    plt.legend()
    plt.show()
    return

In [16]:
# evaluation function to compute rmse
def evaluate(target_var):
    rmse = round(sqrt(mean_squared_error(validate[target_var], yhat_df[target_var])), 0)
    return rmse


In [17]:
# plot and evaluate 
def plot_and_eval(target_var):
    plt.figure(figsize = (12,4))
    plt.plot(train[target_var], label = 'Train', linewidth = 1)
    plt.plot(validate[target_var], label = 'Validate', linewidth = 1)
    plt.plot(yhat_df[target_var])
    plt.title(target_var)
    rmse = evaluate(target_var)
    print(target_var, '-- RMSE: {:.0f}'.format(rmse))
    plt.show()

In [18]:
# Create the empty dataframe
eval_df = pd.DataFrame(columns=['model_type', 'target_var', 'rmse'])

# function to store rmse for comparison purposes
def append_eval_df(model_type, target_var):
    rmse = evaluate(target_var)
    d = {'model_type': [model_type], 'target_var': [target_var], 'rmse': [rmse]}
    d = pd.DataFrame(d)
    return eval_df.append(d, ignore_index = True)